## Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

## Load Dataset

In [2]:
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
dataset = torchvision.datasets.ImageFolder(root="../input/persian-mnist/MNIST_persian",transform=transform)
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset,[train_size,test_size])
train = torch.utils.data.DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=4)
test = torch.utils.data.DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=4)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Transfer Learning

### Resnet152

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.resnet152(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

In [4]:
in_features = model.fc.in_features
model.fc = nn.Linear(in_features,10)
model = model.to(device)
ct = 0
for child in model.children():
    ct += 1
    if ct < 7:
        for param in child.parameters():
            param.requires_grad = False

In [5]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loss_function = nn.CrossEntropyLoss()

In [6]:
def accuracy(pred,label):
    _,pred_max = torch.max(pred,1)
    acc = torch.sum(pred_max==label,dtype=torch.float64) / len(pred)
    return acc

In [7]:
for epoch in range(30):
    train_acc = 0.0
    train_loss = 0.0
    model.train()
    for image, label in tqdm(train):
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        predict = model(image)
        loss = loss_function(predict,label)
        loss.backward()
        optimizer.step()
        
        train_acc += accuracy(predict,label)
        train_loss += loss

    train_total_acc = train_acc / len(train)
    train_total_loss = train_loss / len(train)
    
    print(f"Epochs:{epoch+1}, Loss:{train_total_loss}, acc:{train_total_acc}")

100%|██████████| 34/34 [00:18<00:00,  1.81it/s]


Epochs:1, Loss:0.8278653621673584, acc:0.7478553921568627


100%|██████████| 34/34 [00:11<00:00,  2.89it/s]


Epochs:2, Loss:0.29544803500175476, acc:0.9114583333333333


100%|██████████| 34/34 [00:11<00:00,  3.00it/s]


...

Epochs:28, Loss:0.020741673186421394, acc:0.9941789215686275


100%|██████████| 34/34 [00:11<00:00,  2.91it/s]


Epochs:29, Loss:0.010006648488342762, acc:0.9972426470588235


100%|██████████| 34/34 [00:11<00:00,  2.99it/s]

Epochs:30, Loss:0.00853770598769188, acc:0.9990808823529411


In [10]:
torch.cuda.empty_cache()

In [9]:
test_loss = 0.0
test_acc = 0.0
model.eval()
for image, label in test:
    image, label = image.to(device), label.to(device)
    predict = model(image)
    loss = loss_function(predict,label)
    
    test_loss += loss
    test_acc += accuracy(predict,label)

test_total_acc = test_acc / len(test)
test_total_loss = test_loss / len(test)

print(f"Loss:{test_total_loss}, acc:{test_total_acc}")

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Loss:0.0011762721696868539, acc:1.0


In [11]:
torch.save(model.state_dict(),"weights.pth")